# `DSML_WS_11` - Decision Trees

In this workshop we take a deep-dive on tree-based methods (and ensembles thereof) commonly used in a myriad of classification and regression problems.

We will cover the following: 
1. **Decision Trees for classification**: breast cancer example
1. **Decision Trees for regression**: peak electricity demand example
1. **Ensemble methods**: XGBoost, random forest

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
np.set_printoptions(suppress=True)

%matplotlib inline

---

## 1. Decision Trees for classification: classifying breast cancer cells

In [ ]:
# load dataset
cancer_df = pd.read_csv("breast_cancer.csv", index_col="id")
cancer_df.head()

To abstract from the relatively high-dimensionality of the breast cancer dataset let us confine our analysis to a two-dimensional feature vector consisting of `area_mean` and `concave points_mean` for now.

In [ ]:
def plot_cells():
    plt.figure(figsize=(8,6))
    plt.scatter(cancer_df[cancer_df["diagnosis"]=='M']['area_mean'], cancer_df[cancer_df["diagnosis"]=='M']['concave points_mean'], marker='x', color='C3')
    plt.scatter(cancer_df[cancer_df["diagnosis"]=='B']['area_mean'], cancer_df[cancer_df["diagnosis"]=='B']['concave points_mean'], marker='+', color='C0')
    plt.xlim([0,2600])
    plt.ylim([0,0.21])
    plt.xlabel("Mean Area",fontsize=16)
    plt.ylabel("Mean Concave Points",fontsize=16)
    plt.legend(['Malignant','Benign'],fontsize=12)
    
plot_cells()

We define our X and Y vectors correspondingly:

In [ ]:
X = np.array(cancer_df[['area_mean','concave points_mean']])
Y = cancer_df['diagnosis'].values

# recode Y to 0 and 1
Y  = np.where(Y=="M", int(1), Y) 
Y  = np.where(Y=="B", int(0), Y) 
Y = Y.astype('int')

Note that we do not need to normalize, as Decision Trees do not work based on distances across features!

Let's specify and fit a simple `DecisionTreeClassifier`, which is available via `sklearn`.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

tree_classifier = DecisionTreeClassifier(max_depth=2,criterion='gini') # we set gini as our impurity measure
tree_classifier.fit(X, Y)

The decision estimator has an attribute called tree_  which stores the entire tree structure and allows access to low-level attributes. The binary tree_ attribute is represented as a number of parallel arrays. The i-th element of each array holds information about the node `i`. Node 0 is the tree's root.

In [ ]:
structure = tree_classifier.tree_

**Question**: Recall the basic terminology for decision trees, including nodes, leaves, parent and children nodes, threshold and impurity.

Among those arrays, we have:

   - left_child: id of the left child of the node
   - right_child: id of the right child of the node
   - feature: feature used for splitting the node
   - threshold: threshold value used for splitting the node
   - impurity: the impurity at the node
   - etc.

In [ ]:
# assign various tree attributes
n_nodes = structure.node_count
n_leaves = structure.n_leaves
children_left = structure.children_left
children_right = structure.children_right
feature = structure.feature
threshold = structure.threshold
impurity = structure.impurity

In [ ]:
print("Num nodes: \t",n_nodes)
print("Num leaves: \t",n_leaves)

**Question**: Based on this information, can you draw the basic structure of the tree?

In [ ]:
print("left children per node: ", children_left)
print("right children per node: ", children_right)

**Question**: Based on this information, can you add the node IDs to your basic tree sketch?

In [ ]:
print("Decision feature at node: ", feature)
print("Threshold of feature at node", threshold)
print("Impurity at node: ", impurity)

**Question**: Based on this information, can you derive which feature and threshold was used to split each node?

Next let's retrieve the decision path of a selected sample. 

The `decision_path` method allows us to retrieve the node indicator functions. A non-zero element of an indicator matrix at the position (i, j) indicates that the sample i goes through the node j.

In [ ]:
node_indicators = tree_classifier.decision_path(X) # results in a 569x7 sparse matrix

# let's generate a random sample ID
sample_id = np.random.randint(0,len(X))

# retrieve decision_path for that sample
node_index = node_indicators.indices[node_indicators.indptr[sample_id]: #indptr maps the elements of data and indices to the rows of the sparse matrix
                                    node_indicators.indptr[sample_id + 1]]  #indptr maps the elements of data and indices to the rows of the sparse matrix

print("Decision path for sample " + str(sample_id), ": ", str(node_index))

The `apply` method can be used to get the index of the leaf that each sample is predicted as.

In [ ]:
# we can also retrieve the leaf ids reached by each sample using .apply
leaf_ids = tree_classifier.apply(X)

leaf_ids[:10]

**Question**: For our basic decision tree, what combinations of IDs could appear as decision paths? What IDs could be returned by  `apply()`?

Finally, let us also consider the features and thresholds that were used to predict a certain sample.

In [ ]:
print('Decision path for sample %s: %s' % (str(sample_id), str(node_index)))
print('Feature values of sample %s: %s \n' % (sample_id, X[sample_id]))
print('Rules used to predict sample %s: ' % sample_id)
for node_id in node_index:
    # skip leaf node
    if leaf_ids[sample_id] == node_id:
        continue
    
    # for all other nodes, retrieve the feature values
    if (X[sample_id, feature[node_id]] <= threshold[node_id]):
        threshold_sign = "<="
    else:
        threshold_sign = ">"

    print("Decision at node %s: value for feature %s (%s) is %s the threshold of %s"
          % (node_id,
             feature[node_id],
             X[sample_id, feature[node_id]],
             threshold_sign,
             threshold[node_id]))

### Plot the full decision tree

In [ ]:
from sklearn import tree

plt.figure(figsize=(10,6))
tree.plot_tree(tree_classifier, class_names=['Malignant','Benign'], feature_names=['area_mean', 'concave points_mean'])

In the decision tree each node is represented by a box. For each node the following information is provided:
- decision feature and threshold
- impurity
- number of samples
- number of samples per class
- class (i.e., majority vote)

We can, thus, easily relate this back to the tree attributes we computed above. A selection is below:

In [ ]:
print("Num nodes: \t",n_nodes)
print("Num leaves: \t",n_leaves)
print("Feature at node", feature) # -2 indicates no feature/threshold, i.e. a leaf
print("Threshold of feature at node", threshold)
print("Impurity at node: ", impurity)

### Plot decision surfaces

As we have seen in the lecture, another intuitive representation of decision trees is the use of decision surfaces. These can be related back directly to the decision tree. For ease of use, a plotting routine has been prepared that combines fitting and plotting into a single routine and allows for easy adjustment of tree depth and the minimum samples per leaf (discussed below).

In [ ]:
def plot_class_surface(max_depth,min_samples_leaf=1):
    
    # specify and fit decision tree classifier
    from sklearn.tree import DecisionTreeClassifier, export_graphviz # we also call the garphviz module for later visualization
    model = DecisionTreeClassifier(max_depth=max_depth,
                                   min_samples_leaf=min_samples_leaf,
                                  criterion='gini') # we set entropy as our impurity measure
    model.fit(X, Y)
    
    # get tree attributes
    attributes = model.tree_
    
    # define range per feature
    x_range = [0,2600] # i.e. mean area
    y_range = [0, 0.21] # i.e mean conc. points
    plt.figure(figsize=(8,6))
    
    # plot classification regions
    grid=1000
    xx,yy = np.meshgrid(np.linspace(x_range[0], x_range[1], grid),
                        np.linspace(y_range[0], y_range[1], grid))

    zz = model.predict(np.c_[xx.ravel(), yy.ravel()])
    zz = zz.reshape(xx.shape)
    cs = plt.contourf(xx, yy,zz,levels=[-float("inf"),0,float("inf")],alpha=0.2,colors=["b","r"])
    plt.contour(cs, colors='k')
    
    # plot data points
    s1 = plt.scatter(cancer_df[cancer_df["diagnosis"]=='M']['area_mean'], cancer_df[cancer_df["diagnosis"]=='M']['concave points_mean'], marker='x', color='C3')
    s2 = plt.scatter(cancer_df[cancer_df["diagnosis"]=='B']['area_mean'], cancer_df[cancer_df["diagnosis"]=='B']['concave points_mean'], marker='+', color='C0')    
    plt.xlim([0,2600])
    plt.ylim([0,0.21])
    plt.xlabel("Mean Area",fontsize=16)
    plt.ylabel("Mean Concave Points",fontsize=16)
    plt.legend([s1,s2],['Malignant','Benign'],fontsize=12)
    
    print("number of nodes: ", attributes.node_count)
    print("number of leafs: ", attributes.n_leaves)
    
    plt.show()
    
    #plt.savefig("Breast_Cancer_Decision_Surface_{}depth.pdf".format(tree_depth))

In [ ]:
plot_class_surface(3)

### Controlling overfitting in Decision Trees

**Decision-tree learners can create overly complex trees that do not generalise the data well. This is called overfitting. Mechanisms such as pruning, setting the minimum number of samples required at a leaf node or setting the maximum depth of the tree are necessary to avoid this problem.**

This can easily be seen by increasing tree depth to unreasonable values:

In [ ]:
plot_class_surface(15)

What can we do about overfitting in sklearn? As mentioned, we have several tools at our disposal:
- **max_depth**: The maximum depth of the tree. If None, then nodes are expanded until all the leaves contain less than min_samples_split samples. A too high value of maximum depth causes overfitting, whereas a too low value causes underfitting.
- **min_samples_leaf**: By specifying a minimum number of samples per leaf, overfitting can be controlled for.
- **ccp_alpha**: Cost Complexity (CCP) alpha paramter determining the size of the penalty.

**Question**: Check the effect of the max_depth and the min_samples_leaf parameters. How would you use them to prevent overfitting?

In [ ]:
plot_class_surface(max_depth=15, min_samples_leaf=1)

Let us look at the **cost complexity** as an effective measure in avoiding overfitting. The cost complexity of a tree (CCP(T)) is defined as 

\begin{equation}
CCP(T) = ERR(Z) + \alpha L(T)
\end{equation}

where ERR(Z) is the total misclassification rate of the terminal nodes and L(T) is the number of terminal nodes of tree T. This type of formula should look familiar, as it closely resembles the regularized regression loss functions we know.

To get an idea of what values of $\alpha$ could be appropriate, `scikit-learn` provides `DecisionTreeClassifier.cost_complexity_pruning_path` that returns the effective alphas (i.e., those that will achieve the next step in complexity reduction) and the corresponding total leaf impurities at each step of the pruning process. As alpha increases, more of the tree is pruned, which increases the total impurity of its leaves.

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

# fit decision tree (without limit on max_depth, i.e. tree will grow fully if alpha is set to 0)
tree_classifier = DecisionTreeClassifier(random_state=0, 
                                         criterion="gini")

# compute cost_complexity_pruning_path 
path = tree_classifier.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
#plot cost_complexity_pruning_path
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(ccp_alphas[:-1], impurities[:-1], marker='o', drawstyle="steps-post")  # we remove the last alpha as this corresponds to a tree with only the root node
ax.set_xlabel("effective alpha",fontsize=16)
ax.set_ylabel("total impurity of leaves",fontsize=16)
ax.set_title("Total Impurity vs effective alpha for training set",fontsize=16)
#plt.savefig("Determining_Alpha.pdf")

Next, we train a decision tree using the effective alphas. The last value in ccp_alphas is the alpha value that prunes the whole tree, leaving the tree with one node.

In [ ]:
trees = []
for ccp_alpha in ccp_alphas:
    tree = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    tree.fit(X_train, y_train)
    trees.append(tree)
print("Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
      trees[-1].tree_.node_count, ccp_alphas[-1]))

For the remainder of this example, we remove the last element in clfs and ccp_alphas, because it is the trivial tree with only one node. Here we show that the number of nodes and tree depth decreases as alpha increases.

In [ ]:
trees = trees[:-1]
ccp_alphas = ccp_alphas[:-1]

node_counts = [tree.tree_.node_count for tree in trees]
depth = [tree.tree_.max_depth for tree in trees]
fig, ax = plt.subplots(1,2,figsize=(14,6))
ax[0].plot(ccp_alphas, node_counts, marker='o', drawstyle="steps-post")
ax[0].set_xlabel("alpha",fontsize=16)
ax[0].set_ylabel("number of nodes",fontsize=16)
ax[0].set_title("Number of nodes vs alpha",fontsize=16)
ax[1].plot(ccp_alphas, depth, marker='o', drawstyle="steps-post")
ax[1].set_xlabel("alpha",fontsize=16)
ax[1].set_ylabel("depth of tree",fontsize=16)
ax[1].set_title("Depth vs alpha",fontsize=16)
fig.tight_layout()
#plt.savefig("Pruning_effect.pdf")

In [ ]:
len(ccp_alphas)

Now, we could implement a grid search over the identified effective alphas to determine where predictive performance is maximized (feel free to try this out!).

### Small excursion: Naive Bayes

In last week's lecture, you also learned about the Naive Bayes algorithm. It is based on conditional probabilities that you use to calculate the change in probability of class membership. Say, for example, you have an unknown cell structure and want to calculate how likely it is that this cell belongs to the Malignant cells. You can, then, look at the conditional probability of a cell having that specific mean area to predict whether the unknown cell is malignant.

In [ ]:
# use Naive Bayes from sklearn
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)

point_benign, point_malignant = gnb.theta_ # gives the mean of each feature per class
radius_benign, radius_malignant = np.sqrt(gnb.var_) # gives the SD of each feature per class

In [ ]:
import matplotlib.patches as mpl_patches
benign = mpl_patches.Ellipse(xy=point_benign, width=3 * radius_benign[0], height=3 * radius_benign[1], facecolor='none', edgecolor='b', linewidth=0.5)
malignant = mpl_patches.Ellipse(xy=point_malignant, width=3 * radius_malignant[0], height=3 *radius_malignant[1], facecolor='none', edgecolor='r', linewidth=0.5)
plt.scatter(point_benign[0], point_benign[1], color='b')
plt.scatter(point_malignant[0], point_malignant[1], color='r')

ax = plt.gca()
ax.add_artist(benign)
ax.add_artist(malignant)

s1 = plt.scatter(cancer_df[cancer_df["diagnosis"]=='M']['area_mean'], cancer_df[cancer_df["diagnosis"]=='M']['concave points_mean'], marker='x', color='C3', linewidths=0.2)
s2 = plt.scatter(cancer_df[cancer_df["diagnosis"]=='B']['area_mean'], cancer_df[cancer_df["diagnosis"]=='B']['concave points_mean'], marker='+', color='C0', linewidths=0.2)    
plt.xlim([0,2600])
plt.ylim([0,0.21])
plt.xlabel("Mean Area",fontsize=16)
plt.ylabel("Mean Concave Points",fontsize=16)
plt.legend([s1,s2],['Malignant','Benign'],fontsize=12)

You could now evaluate and compare both models using the known cross-validation procedure and classification evaluation metrics.

---

## 2. Decision Trees for regression: predicting peak electricity demand

We continue with our electric power example from last week which we retieved from PJM from the following link [here](https://dataminer2.pjm.com/feed/hrl_load_metered/definition). The files we are loading are the raw files we downloaded from this source. The final input data for our code is `Pittsburgh_load_data.csv`.

In [ ]:
df = pd.read_csv("Pittsburgh_load_data.csv")
df["Date"] = pd.to_datetime(df["Date"], format="%d.%m.%Y")
df["Month"] = df["Date"].apply(lambda x: x.month)
df.head()

In [ ]:
# define x and y vectors
Xp = df["High_temp"].values
Yp = df["MAX"].values

In [ ]:
plt.figure(figsize = (8,6))
plt.scatter(Xp, Yp, marker="x")
plt.xlabel("High Temperature (°C)")
plt.ylabel("Peak Demand (GW)")
plt.show()

We have already shortly touched upon decision trees for regression in Workshop 8. But let's quickly revisit. We will use the `DecisionTreeRegressor` class in `scikitlearn`to fit and plot a decision tree regressor.

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

def plot_tree_regression_line(tree_depth):
    # fit regression model (to full data)
    Tree_reg = DecisionTreeRegressor(max_depth=tree_depth,criterion="squared_error")
    Tree_reg.fit(Xp.reshape((-1,1)), Yp) 
    
    attributes = Tree_reg.tree_

    # plot
    plt.figure(figsize = (8,6))
    plt.scatter(Xp, Yp, marker="x")
    plt.plot(np.arange(-18,40,1), Tree_reg.predict(np.arange(-18,40,1).reshape((-1,1))), marker="x", color='C1')
    plt.xlabel("High Temperature (°C)", fontsize=16)
    plt.ylabel("Peak Demand (GW)", fontsize=16)
    
    print("number of nodes: ", attributes.node_count)
    print("number of leafs: ", attributes.n_leaves)
    
    #plt.savefig("Peak_Power_Regression_Lines_{}depth.pdf".format(tree_depth))
    return Tree_reg

In [ ]:
tree_reg = plot_tree_regression_line(tree_depth = 3)

In [ ]:
plt.figure(figsize = (12,6))
plot_tree(tree_reg, feature_names=['High Temperature'], class_names=['Peak Demand'])

---

## 3. Ensemble Methods

In predictive modeling, “risk” is equivalent to variation (i.e. variance) in prediction error. Ensemble methods are targeted at reducing variance, thus increasing predictive power.
The core idea is that by combining the outcomes of individual models, e.g., by taking an average, variance may be reduced. Thus, using an average of two or more predictions can potentially lead to smaller error variance, and therefore better predictive power.

We will not discuss ensemble methods in detail here, but will limit our discussion to a brief introduction of two very popular tree-based ensemble methods. These are:

- XGBoost (a type a boosting method): see [here](https://xgboost.readthedocs.io/en/latest/)
- RandomForest: see [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) and [here](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

### XGBoost

XGBoost is an ensemble method that uses **boosting**. While XGBoost is not included in sklearn, there is a very well developed API that can be installed by executing the following command:
- `conda install -c conda-forge xgboost`

Once you have completed the installation you are good to go. Let us fit a very simple classifier to the breast cancer dataset.

In [ ]:
# train test split on breast cancer dataset
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=42)

In [ ]:
# specify and fit model
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(booster="gbtree")
xgb_classifier.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test,xgb_classifier.predict(X_test))

In [ ]:
accuracy_score(y_test,xgb_classifier.predict(X_test))

Obviously, there is likely room for improvement as you grid search some of the hyperparameters. However, by just taking the default setting, we already achieve an accuracy score that is comparable to that of the grid-searched decision tree above.

### Random Forest

Random Forests is a selection of n trees which are trained in parallel. Predictions are made by averaging the outputs across these n trees. Random Forest are most often combined with **bagging**, i.e. different boostrap samples of the training data are used to train the individual trees.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# sepcify and fit model
rf_classifier = RandomForestClassifier(n_estimators=100, 
                                       bootstrap=True, random_state=42) # we select boostrapp, i.e. we use bagging
rf_classifier.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test,rf_classifier.predict(X_test))

In [ ]:
accuracy_score(y_test,rf_classifier.predict(X_test))

Again, just by taking the default setting, we obtain very good results that are comparable to those of the fully grid-searched decision tree.

---